In [2]:
import torch
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification, AutoModelForCausalLM, TFAutoModelForCausalLM

In [3]:
model_gpt1 =  'sberbank-ai/rugpt3small_based_on_gpt2'
model_gpt2 = 'cointegrated/rubert-tiny'
device = torch.device("cpu")
tokenizer_dialog = AutoTokenizer.from_pretrained(model_gpt1)
model_dialog = AutoModelForCausalLM.from_pretrained(model_gpt1).to(device)

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [4]:
import math
import string

In [21]:
def calculate_perplexity(sentence, model, tokenizer):
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    range_max = 400
    # print(sentence)
    sentence_positive = 'довольна: ' + sentence[:range_max]
    sentence_negative = 'недовольна: ' + sentence[:range_max]
    #sentence_positive = 'хороший:'+ sentence[:range_max]
    #sentence_negative = 'плохой:'+ sentence[:range_max]
    # sentence_positive = 'интересный:'+ sentence[:range_max]
    # sentence_negative = 'скучный:'+ sentence[:range_max]
    list_sent = [sentence_positive, sentence_negative]
    ppl_values = []
    flag_non_calc = False

    for sent in list_sent:
        encodings = tokenizer(sent, return_tensors='pt')
        input_ids = encodings.input_ids.to(device)
        with torch.no_grad():
            outputs = model(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss
        multiple_loss = loss.item() * input_ids.size(1)
        if multiple_loss > 500:
            flag_non_calc = True
            break
        ppl = math.exp(multiple_loss)
        ppl_values.append(ppl)

    if flag_non_calc:
        result = 'не определено'
    else:
        if ppl_values[0] > ppl_values[1]:
            result = 'отрицательный'
        elif ppl_values[0] < ppl_values[1]:
            result = 'положительный'
    # print(result)
    return result

In [6]:
import pandas as pd

In [7]:
df = pd.read_pickle('doramy_comments_plain.pkl')

In [12]:
df.head(1000)

,id,author,time,comment
0,34127,Leodo,16 ноября 2022,"\nОчень интригующее начало, а судя по персонаж..."
1,34127,Nilufar,16 ноября 2022,\nОчень интересная дорома\n
2,34127,Dariia13,16 ноября 2022,\nЯ все жду «Децибел» с Ким Ре Воном и Ли Чон ...
3,34127,Маркус,16 ноября 2022,\nчто то хромать стали выходы серий дорам с оз...
4,34127,Лиза Мин,16 ноября 2022,\nНачало очень интригует! Ещо одна дорама в ко...
...,...,...,...,...
19,34086,Moon,12 ноября 2022,\nВаввв!обов’язково до перегляду💞\n
20,34086,ответ,12 ноября 2022,"\nглавный герой псих, вау\n"
21,34086,Аяулым,12 ноября 2022,\nКласный дорама\n
22,34086,ретти,12 ноября 2022,"\nХотя и вышло всего 2 серии, меня эта дорама ..."


In [10]:
df.loc[df.id == '34127','comment']

0     \nОчень интригующее начало, а судя по персонаж...
1                           \nОчень интересная дорома\n
2     \nЯ все жду «Децибел» с Ким Ре Воном и Ли Чон ...
3     \nчто то хромать стали выходы серий дорам с оз...
4     \nНачало очень интригует! Ещо одна дорама в ко...
5                                   \nИнтересненько 👍\n
6     \nОчень интересная дорама жду продолжения.\nСп...
7     \nВсё очень сочно и классно звучит!!\nНапомина...
8                       \nЭто лучшее что я смотрела 🔥\n
9     \nДовольно интересный сюжет. Как раз по эти жа...
10                  \nЧто ж, выглядит многообещающе))\n
11                        \nнеплохо, довольно неплохо\n
12    \nДорама очень интересная и отличная. Эта дора...
13                    \nИнтересный дорама про полицию\n
14                 \nОчень крутая дорама всем советую\n
15                                  \nПока что смотрю\n
16            \nДорама бомба,честно говоря!!!!!!!!!!!\n
17                              \nОчень понравил

In [16]:
df.loc[df.id == '34127','comment'].apply(lambda x: calculate_perplexity(x, model_dialog, tokenizer_dialog))

0     положительный
1     положительный
2     положительный
3     положительный
4     положительный
5     положительный
6     положительный
7     положительный
8     положительный
9     положительный
10    положительный
11    положительный
12    положительный
13    положительный
14    положительный
15    положительный
16    положительный
17    положительный
Name: comment, dtype: object

In [26]:
x = ' мне не понравилось,скучновато\n'
# x = 'отвратительный дизайн'
calculate_perplexity(x, model_dialog, tokenizer_dialog)

'отрицательный'